In [ ]:
import pandas as pd

# Cleaning the csv dataset

objective: create a dataframe where there is 1 gtd_id, 1 headline, 1 link

Cleaning the 2_GTD_case_details_2015_2017.csv dataset into id,link

let us have 2 dataframe s_df and df2

In [ ]:
import csv
from IPython.display import display

df =pd.read_csv('2_GTD_case_details_2015_2017.csv')

#return headline,news source,date, year
def strip_src(src):
    if type(src)!=float:
        b = ast.literal_eval(src)
        headline = re.findall(r'"([^"]*)"',b[0])[0]

        lst = b[0].replace('\"','')
        lst = re.split(",",lst)
        dates = lst[-2:]
        dates =','.join(dates).strip()
        source = str(lst[-3]).strip()
        df = pd.DataFrame([headline, source, dates]).transpose()
        df.columns=['headline','source','date']
    else:
        df = pd.DataFrame(columns=['headline','source','date'])
    return df

lst = ['1_src_0', '1_src_1', '1_src_2']

s_df = pd.DataFrame()

def unstack_A(df,a):
    scrape_df = pd.DataFrame()
    for i in range(len(df)):
        gtd_id = pd.DataFrame([df['GTD ID:'].iloc[i]])
        gtd_id.columns=['GTD ID:']
        sdf = strip_src(df[a].iloc[i])
        sdf = pd.concat([gtd_id,sdf],axis=1)
        scrape_df = pd.concat([scrape_df,sdf])
    return scrape_df

for i in lst:
    sdf=unstack_A(df,i)
    s_df=pd.concat([s_df,sdf])
s_df = s_df.dropna()
s_df

cleaning the google links sheet into something headline, link

In [ ]:
df2=pd.DataFrame()
counter = 0
with open('0.1_1_google_link_3.csv', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        counter+=1
        print(counter,end='\r')
        dfa = pd.DataFrame(row).transpose()
        df2 = pd.concat([dfa,df2])

df2 = df2.drop_duplicates().drop(columns=[0,2,3]).reset_index(drop=True)

In [ ]:
df2

In [ ]:
main_df = pd.DataFrame()
for i in range(len(df2)):
    #print(i)
    df3=pd.DataFrame(df2.iloc[i]).transpose()
    df3 = pd.melt(df3,id_vars=[1])
    main_df = pd.concat([main_df,df3])

In [ ]:
main_df = main_df.dropna().reset_index(drop=True)
main_df.columns=['headline','variable','link']
main_df

In [ ]:
display(main_df.head(), s_df)

In [ ]:
#main_df.merge(s_df, left_on='headline', right_on='headline', how='outer').sort_values(by=['headline'])
#pd.merge(main_df,s_df)

import numpy as np

d1=main_df[['headline','link']]
d2=s_df[['GTD ID:','headline']]

d1['link'].replace('', np.nan, inplace=True)
d1=d1.dropna(subset=['link']).reset_index(drop=True)
d2['headline'].replace('', np.nan, inplace=True)
d2=d2.dropna(subset=['headline']).reset_index(drop=True)

#because d2 headline has an extra comma, we will clean it here
d2['headline']=d2['headline'].apply(lambda x: x[:-1])

display(d1,d2)


#for i in range(len(d1)):
    
#d1.to_csv('0.1_1_google_link_4.csv')
df4 = pd.merge(d1,d2, on='headline')
display(df4)
df4.to_csv('0.1_1_google_link_4.csv')
df4

In [ ]:
df4.reset_index(drop=True)

# Link collecting

In [2]:
import pandas as pd
import asyncio
import aiohttp
import ssl

from bs4 import BeautifulSoup
from collections import Counter
import re
from urllib.parse import urlparse
import time
import ast


import requests

from IPython.display import display


#conda install tornado==4.5.3 as tornado 5x is not good for this

#asynschronously fetch one url
async def fetch(session, url):
    async with session.get(url, ssl=ssl.SSLContext()) as response:
        return await response.text()

#asynschronously fetch multiple    
async def fetch_all(urls, loop):
    async with aiohttp.ClientSession(loop=loop) as session:
        results = await asyncio.gather(*[fetch(session, url) for url in urls], return_exceptions=True)
        return results

#function to return a soup object
def soup(page):
    try:
        soup = BeautifulSoup(page, 'html.parser')
        return soup
    except:
        soup = []
        return soup

#function to return a list of texts in divs
def div_soup(page):
    try:
        soup = BeautifulSoup(page, 'html.parser')
        tags = soup.find_all('div')
        tags_list = [t.text for t in tags]
        return tags_list
    except:
        tags_list = []
        return tags_list    
    
#function to return a list of soup objects
def soup_list(urls):
    loop = asyncio.new_event_loop()
    htmls = loop.run_until_complete(fetch_all(urls, loop))
    soup_list = list(map(soup,htmls))
    loop.close()
    return soup_list

df=pd.read_csv('0.1_1_google_link_4.csv').dropna().drop(['Unnamed: 0'],axis=1).reset_index(drop=True)
df_links = df.drop_duplicates(subset=['link'])
df_links.head()

,headline,link,GTD ID:
0,4 civilians killed in ambush by suspected BIFF...,https://newsinfo.inquirer.net/699492/police-of...,201506180097
1,"Trooper killed, 2 wounded in Davao Del Sur cla...",https://en.wikipedia.org/wiki/Battle_of_Marawi,201506190018
2,"Trooper killed, 2 wounded in Davao Del Sur cla...",https://www.google.com.sg/search?source=hp&ei=...,201506190018
3,"Trooper killed, 2 wounded in Davao Del Sur cla...",https://en.wikipedia.org/wiki/Timeline_of_the_...,201506190018
4,"Trooper killed, 2 wounded in Davao Del Sur cla...",https://www.crisisgroup.org/crisiswatch/print?...,201506190018


execution

In [3]:
total = 13594
a = 10000
b = 10005
counter = a

url_list = [i for i in df_links['link']]
id_list = [i for i in df_links['GTD ID:']]
#display(url_list,id_list)


#this code runs through the df_links list, returns a h5 file with 1 website and all its divs in text form.
while b <= 13590:
    loop = asyncio.new_event_loop()
    print('Getting ' +str(a) + ' to ' +str(b))
    urls = url_list[a:b]
    htmls = loop.run_until_complete(fetch_all(urls, loop))
    #print(htmls[-1])
    soups = list(map(div_soup,htmls))
    lst = list(zip(url_list,id_list,soups))
    l_df = pd.DataFrame(lst)
    #df2 = pd.concat([df2,l_df])
    print(type(l_df[2][0]))
    display(l_df)
    
    for i in range(len(l_df)):
        gtd_id = l_df[1][i]
        l_df.iloc[i].to_pickle('{}_{}.h5'.format(gtd_id,counter))
        counter +=1

    a += 5
    b += 5

    loop.close()


Getting 10000 to 10005
<class 'list'>


,0,1,2
0,https://newsinfo.inquirer.net/699492/police-of...,201506180097,[]
1,https://en.wikipedia.org/wiki/Battle_of_Marawi,201506190018,[]
2,https://www.google.com.sg/search?source=hp&ei=...,201506190018,[]
3,https://en.wikipedia.org/wiki/Timeline_of_the_...,201506190018,"[\n function setAttributeOnload(object, att..."
4,https://www.crisisgroup.org/crisiswatch/print?...,201506190018,[]


Getting 10005 to 10010
<class 'list'>


,0,1,2
0,https://newsinfo.inquirer.net/699492/police-of...,201506180097,"[\n function setAttributeOnload(object, att..."
1,https://en.wikipedia.org/wiki/Battle_of_Marawi,201506190018,"[, \n\n\n\n\n\n\n\n\n\nPhilstar HomeThe Philip..."
2,https://www.google.com.sg/search?source=hp&ei=...,201506190018,"[, \n\n\n\n\n\n\n\n\n\nPhilstar HomeThe Philip..."
3,https://en.wikipedia.org/wiki/Timeline_of_the_...,201506190018,[\n Bomb blast at Philippine shopping...
4,https://www.crisisgroup.org/crisiswatch/print?...,201506190018,[]


Getting 10010 to 10015
<class 'list'>


...

,0,1,2
0,https://newsinfo.inquirer.net/699492/police-of...,201506180097,[\nThailand Chatter Forum > News Flash > The N...
1,https://en.wikipedia.org/wiki/Battle_of_Marawi,201506190018,"[\n function setAttributeOnload(object, att..."
2,https://www.google.com.sg/search?source=hp&ei=...,201506190018,"[\n function setAttributeOnload(object, att..."
3,https://en.wikipedia.org/wiki/Timeline_of_the_...,201506190018,[\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
4,https://www.crisisgroup.org/crisiswatch/print?...,201506190018,[\n\n \n if (show_overlay==1 || (dfp_pre...


Getting 13580 to 13585
<class 'list'>


,0,1,2
0,https://newsinfo.inquirer.net/699492/police-of...,201506180097,[SectionsSEARCHSkip to contentSkip to site ind...
1,https://en.wikipedia.org/wiki/Battle_of_Marawi,201506190018,"[\n\n\n\nGoSearch\n\n\n\n, \n\n\nGoSearch\n\n\..."
2,https://www.google.com.sg/search?source=hp&ei=...,201506190018,"[\n\n\n\n\n\n\n, \n\n\n\n\n, \n\n\n, \n, \n\n\..."
3,https://en.wikipedia.org/wiki/Timeline_of_the_...,201506190018,[\n\n\nMyNewMarkets.com\nClaims Journal\nInsur...
4,https://www.crisisgroup.org/crisiswatch/print?...,201506190018,[]


Getting 13585 to 13590
<class 'list'>


,0,1,2
0,https://newsinfo.inquirer.net/699492/police-of...,201506180097,[]
1,https://en.wikipedia.org/wiki/Battle_of_Marawi,201506190018,[\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\...
2,https://www.google.com.sg/search?source=hp&ei=...,201506190018,[BIGtheme.net http://bigtheme.net/ecommerce/op...
3,https://en.wikipedia.org/wiki/Timeline_of_the_...,201506190018,[]
4,https://www.crisisgroup.org/crisiswatch/print?...,201506190018,"[\nFor faster navigation, this Iframe is prelo..."


In [ ]:
pd.read_pickle('201506190018_2.h5')[2]
